In [1]:
import pandas as pd

import numpy as np

## Acceso a datos mediante ficheros en Python

In [3]:
accidentes = pd.read_csv('./2019_Accidentalidad.csv',
                         delimiter=';',
                         encoding='latin1',
                         parse_dates=['FECHA'],
                         na_values=['-', 'DESCONOCIDA', 'DESCONOCIDO'])

accidentes.to_parquet('./accidentes1.parquet')

## Acceso a datos mediante API públicas en Python

In [4]:
import requests

In [5]:
request_header = {'Accept':'application/vnd.github.v3+json'}

url_base = 'https://api.github.com/search/repositories?'

query = 'q=topic:data-analysis+language:Python&sort=stars&order=desc'

response = requests.get(url_base + query, headers = request_header)


librerias = pd.DataFrame(response.json()['items'])

librerias.head()

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,score
0,843222,MDEwOlJlcG9zaXRvcnk4NDMyMjI=,scikit-learn,scikit-learn/scikit-learn,False,"{'login': 'scikit-learn', 'id': 365630, 'node_...",https://github.com/scikit-learn/scikit-learn,scikit-learn: machine learning in Python,False,https://api.github.com/repos/scikit-learn/scik...,...,True,False,False,"[data-analysis, data-science, machine-learning...",public,24840,2261,56327,main,1.0
1,858127,MDEwOlJlcG9zaXRvcnk4NTgxMjc=,pandas,pandas-dev/pandas,False,"{'login': 'pandas-dev', 'id': 21206976, 'node_...",https://github.com/pandas-dev/pandas,Flexible and powerful data analysis / manipula...,False,https://api.github.com/repos/pandas-dev/pandas,...,True,False,False,"[alignment, data-analysis, data-science, flexi...",public,16885,3692,40231,main,1.0
2,204086862,MDEwOlJlcG9zaXRvcnkyMDQwODY4NjI=,streamlit,streamlit/streamlit,False,"{'login': 'streamlit', 'id': 45109972, 'node_i...",https://github.com/streamlit/streamlit,Streamlit — A faster way to build and share da...,False,https://api.github.com/repos/streamlit/streamlit,...,True,False,False,"[data-analysis, data-science, data-visualizati...",public,2520,687,28294,develop,1.0
3,162405963,MDEwOlJlcG9zaXRvcnkxNjI0MDU5NjM=,gradio,gradio-app/gradio,False,"{'login': 'gradio-app', 'id': 51063788, 'node_...",https://github.com/gradio-app/gradio,Build and share delightful machine learning ap...,False,https://api.github.com/repos/gradio-app/gradio,...,True,False,False,"[data-analysis, data-science, data-visualizati...",public,1647,464,23220,main,1.0
4,283046497,MDEwOlJlcG9zaXRvcnkyODMwNDY0OTc=,airbyte,airbytehq/airbyte,False,"{'login': 'airbytehq', 'id': 59758427, 'node_i...",https://github.com/airbytehq/airbyte,Data integration platform for ELT pipelines fr...,False,https://api.github.com/repos/airbytehq/airbyte,...,True,False,False,"[airbyte, bigquery, change-data-capture, data,...",public,3181,5094,12202,master,1.0


##  Acceso a datos públicos mediante web scraping

In [6]:
from bs4 import BeautifulSoup

import requests

URL = 'https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html'

pagina = requests.get(URL)

soup = BeautifulSoup(pagina.content, 'html.parser')


# Inspeccionando el HTML de la página vemos que las cotizaciones aparecen
# en un elemento de tipo table con el id 'listado_valores'

tabla = soup.find(id = 'listado_valores')


# Dentro de esa tabla hay un elemento de tipo thead con los títulos,
# que seleccionamos para dar nombre a las columnas del dataframe

columnas = [th.text.strip() for th in tabla.find('thead').find_all('th')]


# Dentro de esa tabla los datos están organizados en filas (td) y columnas (tr)
# La primera fila está vacía y la descartamos

datos = [[td.text for td in tr.find_all('td')] for tr in tabla.find_all('tr')[1:]]


# Creamos un dataframe con los datos extraídos

cotizaciones = pd.DataFrame(datos, columns=columnas)

cotizaciones.head()

,Valor,Último,Var. %,Var.,Ac. % año,Máx.,Mín.,Vol.,Capit.,Hora,
0,ACCIONA,"127,400","0,16","0,20","-23,63","127,850","126,350",21.661,6.989,11:46,
1,ACCIONA ENER,"27,580","0,00","0,00","-22,00","27,900","27,400",56.962,9.081,11:46,
2,ACERINOX,"9,694","0,33","0,03","11,34","9,784","9,652",389.950,2.417,11:46,
3,ACS,"33,490","-1,18","-0,40","33,63","33,890","33,460",35.979,9.316,11:46,
4,AENA,"145,950","0,45","0,65","28,41","147,150","145,350",18.414,21.892,11:46,


## Ejemplo de estandarización de datos

In [10]:
# Leemos los datos anteriormente guardados, y descartamos la última columna

accidentes = pd.read_parquet('./accidentes1.parquet')

accidentes = accidentes.iloc[:, :-1]


# Decodificamos la hora y la añadimos a la fecha para tener un campo
# de tipo fecha-hora, más fácil de utilizar

accidentes['FECHA'] += pd.to_timedelta(accidentes.HORA + ':00')


# Eliminamos columnas que ya no son necesarias

accidentes = accidentes.drop(columns=['HORA', 'CALLE', 'NÚMERO'])


# Convertimos todas las columnas a partir de la tercera en tipos categóricos
# (o sea, con un número predefinido de opciones posibles)

accidentes.iloc[:,2:] = accidentes.iloc[:,2:].astype('category')


# Generamos un nuevo campo que codiﬁque los niveles de gravedad en función de
# la "lesividad", según se especiﬁca en el documento de descripción de los datos

c_gravedad = pd.api.types.CategoricalDtype(categories = ['Ileso', 'Leve', 'Grave', 'Fallecido'], ordered=True)

dict_gravedad = {14.0: 'Ileso', 3.0: 'Grave', 4.0: 'Fallecido'}

accidentes['GRAVEDAD'] = accidentes['lesividad*'].apply(lambda x: dict_gravedad.get(x,'Leve') if ~np.isnan(x) else 'Ileso').astype(c_gravedad)


# Guardamos el resultado para análisis posteriores

accidentes.to_parquet('./accidentes2.parquet')

/var/folders/cf/7nzhdlvs5nl0_zh6ky9hz9vc0000gn/T/ipykernel_4284/3622968138.py:11: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  accidentes['FECHA'] += pd.to_timedelta(accidentes.HORA + ' :00')


TypeError: unsupported operand type(s) for +: 'Timedelta' and 'str'

## Ejemplo de ajuste de la granularidad

In [9]:
# Esta función devuelve un dataframe con los datos ya consolidados para un fichero concreto

def get_file(name):
    
    df = pd.read_csv(name, sep=';', parse_dates=['fecha'])

    return df.groupby(['fecha', 'tipo_elem'])\
        .agg({'intensidad' : np.sum, 'ocupacion' : np.mean, 'carga' : np.mean})


# Generamos los nombres de fichero para cada mes y concatenamos los datos

df = pd.concat((get_file(f'./{n:02d}-2019.zip') for n in range(1,13)))


# Guardamos los datos para su uso posterior

df.reset_index().to_parquet('trafico1.parquet')

FileNotFoundError: [Errno 2] No such file or directory: './01-2019.zip'

## Ejemplo de generación de features

In [ ]:
# Leemos los datos de accidentes limpios anteriormente guardados

accidentes = pd.read_parquet('./accidentes2.parquet')


# Calculamos el número de personas implicadas en accidentes por
# tipo de vehículo y por gravedad (solo conductores, no pasajeros)

grav = accidentes[accidentes['TIPO PERSONA'] == 'Conductor']\
    .groupby(['TIPO VEHÍCULO','GRAVEDAD'])['Nº EXPEDIENTE'].nunique()


# Calculamos el porcentaje de graves, leves, etc. para el total de
# los accidentes y para cada tipo de vehículo

grav_rel = grav.groupby('GRAVEDAD').sum() / grav.sum()
grav_rel_veh = grav / grav.groupby('TIPO VEHÍCULO').sum()


# Generamos un indicador que compara la gravedad para los accidentados
# por en tipo de vehículo con la gravedad general

grav_ratio = (grav_rel_veh / grav_rel) - 1


# Seleccionamos los nueve tipos de vehículos más comunes y mostramos los datos
# en una tabla

top_accidentes = accidentes['TIPO VEHÍCULO']\
.value_counts().head(9).index.values
grav_ratio[top_accidentes].unstack(1).loc[top_accidentes].style.format("{:.0%}")

In [ ]:
import altair as alt


# Leemos los datos de accidentes limpios anteriormente guardados

trafico = pd.read_parquet('./trafico1.parquet')


# Generamos la media del indicador de ocupación de las calles en función
# de dos variables sintéticas: la hora del día y el día de la semana

data = trafico.groupby([trafico.fecha.dt.hour, trafico.fecha.dt.dayofweek])\
    ['ocupacion'].mean().rename_axis(['HORA', 'DIA DE LA SEMANA']).reset_index()


# Creamos un sencillo heatmap para mostrar los resultados

alt.Chart(data).mark_rect().encode(
    x='HORA:O',
    y='DIA DE LA SEMANA:O',
    color=alt.Color('ocupacion:Q', legend = None))\
.properties(title='Densidad de tráfico en Madrid (2019)')

In [ ]:
# Leemos los datos de accidentes y de tráfico

accidentes = pd.read_parquet('./accidentes2.parquet')

trafico = pd.read_parquet('./trafico1.parquet')


# Generamos indicadores de accidentalidad y de intensidad de tráfico

data_a = accidentes.groupby([accidentes.FECHA.dt.hour, accidentes.FECHA.dt.dayofweek])\

    ['Nº EXPEDIENTE'].count().rename_axis(['HORA', 'DIA DE LA SEMANA'])


data_t = trafico[trafico['tipo_elem'] == 'URB']\

    .groupby([trafico.fecha.dt.hour, trafico.fecha.dt.dayofweek])\

    ['carga'].mean().rename_axis(['HORA', 'DIA DE LA SEMANA'])


# Con los datos alineados, hacemos el ratio entre los dos indicadores

data = (data_a / data_t).rename('Accidentalidad').reset_index()


# Creamos un sencillo heatmap para mostrar los resultados

alt.Chart(data).mark_rect().encode(
    x='HORA:O',
    y='DIA DE LA SEMANA:O',
    color=alt.Color('Accidentalidad:Q',
        scale=alt.Scale(scheme="lightorange"),legend = None)
    ).properties(title='Relación entre accidentalidad y carga de trafico')